In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import cv2
import torch
import torch.nn.functional as F
from torchvision import transforms
import time
import numpy as np
import os

In [7]:

import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
    
print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


In [ ]:
# Load pretrained model
def load_model():
    try:
        model = torch.hub.load('pytorch/vision:v0.6.0', 'deeplabv3_resnet101', weights='DeepLabV3_ResNet101_Weights.DEFAULT', force_reload=True)
        print("Model has been loaded successfully.")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")

model = load_model()

# Segment people only for the purpose of human silhouette extraction
people_class = 15

# Evaluate model
model.eval()

blur = torch.FloatTensor([[[[1.0, 2.0, 1.0], [2.0, 4.0, 2.0], [1.0, 2.0, 1.0]]]]) / 16.0

# Use GPU if supported, for better performance
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    model.to(device)
    blur = blur.to(device)

# Apply preprocessing (normalization)
preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to create segmentation mask
def makeSegMask(img):
    # Scale input frame
    frame_data = torch.FloatTensor(img) / 255.0
    input_tensor = preprocess(frame_data.permute(2, 0, 1))
    
    # Create mini-batch to be used by the model
    input_batch = input_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_batch)['out'][0]

    segmentation = output.argmax(0)

    bgOut = output[0:1][:][:]
    a = (1.0 - F.relu(torch.tanh(bgOut * 0.30 - 1.0))).pow(0.5) * 2.0

    people = segmentation.eq(torch.ones_like(segmentation).long().fill_(people_class)).float()
    people.unsqueeze_(0).unsqueeze_(0)
    
    for i in range(3):
        people = F.conv2d(people.to(device), blur, stride=1, padding=1)

    # Activation function to combine masks - F.hardtanh(a * b)
    combined_mask = F.relu(F.hardtanh(a * (people.squeeze().pow(1.5))))
    combined_mask = combined_mask.expand(1, 3, -1, -1)

    res = (combined_mask * 255.0).cpu().squeeze().byte().permute(1, 2, 0).numpy()
    return res

# Define directories
input_dir = '/kaggle/input/original-data/Original_data'  # Update with your Kaggle input directory path
output_dir = '/kaggle/working/data_silhouette'  # Update with your Kaggle output directory path

# Check if input directory exists
if not os.path.exists(input_dir):
    print(f"Input directory {input_dir} does not exist.")
else:
    print(f"Input directory {input_dir} exists.")

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process videos
for subdir in os.listdir(input_dir):
    subdir_path = os.path.join(input_dir, subdir)
    if os.path.isdir(subdir_path):
        output_subdir = os.path.join(output_dir, subdir)
        
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        for filename in os.listdir(subdir_path):
            if filename.endswith('.mp4'):
                input_path = os.path.join(subdir_path, filename)
                output_path = os.path.join(output_subdir, f'output-{filename}')

                print(f"Processing video: {input_path}")

                # Loads video file into CV2
                video = cv2.VideoCapture(input_path)
                
                # Check if video opened successfully
                if not video.isOpened():
                    print(f"Error opening video file {input_path}")
                    continue
                
                # Get video file's dimensions
                frame_width = int(video.get(3))
                frame_height = int(video.get(4))
                
                # Creates output video file with XVID codec
                out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

                prev_frame_time = 0
                new_frame_time = 0

                while video.isOpened():
                    # Read each frame one by one
                    success, img = video.read()
                    
                    # Run if there are still frames left
                    if success:
                        # Apply background subtraction to extract foreground (silhouette)
                        mask = makeSegMask(img)
                        
                        # Apply thresholding to convert mask to binary map
                        ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

                        # Write processed frame to output file
                        out.write(thresh)
                        
                        new_frame_time = time.time()
                        fps = 1 / (new_frame_time - prev_frame_time)
                        prev_frame_time = new_frame_time
                        fps = str(fps)
                        print(f"Processing {filename} at {fps} FPS")
                    # Break when there are no more frames  
                    else:
                        break

                # Release resources
                video.release()
                out.release()

print("Processing complete.")


In [18]:
import shutil

output_dir = '/kaggle/working/data_silhouette'
shutil.make_archive('/kaggle/working/data_silhouette', 'zip', output_dir)
print("Zipping complete.")


Zipping complete.
